In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
def data_capturer(path):
  df = pd.read_excel(path, sheet_name=0, usecols="BE:BW", skiprows=8, nrows=45)
  rowTags = pd.read_excel(path , sheet_name=0 , usecols='A' , skiprows = 8 , nrows = 45)
  df = pd.concat([rowTags , df] , axis = 1)
  dictoflist = {}
  for i , r in df.iterrows():
    dictoflist[r[0]]= r[1:]
  df = pd.DataFrame(dictoflist).reset_index()
  dftrain = df.iloc[:14,:]
  dftest = df.iloc[14:,:]
  return df , dftrain , dftest
#df , dftrain , dftest = data_capturer("/content/torbali_veriler.xls")

In [ ]:


def create_sequences(data, seq_length):
    """
    Create sequences for LSTM input
    """
    xs, ys = [], []
    for i in range(len(data)-seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

def mean_absolute_percentage_error(y_true, y_pred):
    """
    Calculate Mean Absolute Percentage Error (MAPE)
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Avoid division by zero
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def build_lstm_model(series):
    """
    Build an LSTM model for univariate time series forecasting

    Parameters:
    series (pd.Series): Input time series data

    Returns:
    dict: Dictionary containing model, predictions, and evaluation metrics
    """
    # Convert to numpy array
    data = series.values.reshape(-1, 1)

    # Split the data into train and test sets
    train = data[:14]
    test = data[14:]

    # Normalize the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_scaled = scaler.fit_transform(train)
    test_scaled = scaler.transform(test)

    # Define sequence length (lookback period)
    seq_length = 3

    # Create sequences
    X_train, y_train = create_sequences(train_scaled, seq_length)

    # Reshape for LSTM [samples, time steps, features]
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(seq_length, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # Train the model
    model.fit(X_train, y_train, epochs=100, verbose=0)

    # Prepare test data for prediction
    # We need to include the last seq_length points from the training set
    test_input = np.concatenate((train_scaled[-seq_length:], test_scaled))
    X_test, y_test_true = create_sequences(test_input, seq_length)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    # Make predictions
    y_test_pred_scaled = model.predict(X_test, verbose=0)

    # Inverse transform the predictions
    y_test_pred = scaler.inverse_transform(y_test_pred_scaled)

    # Inverse transform the actual values
    y_test_true_inv = scaler.inverse_transform(y_test_true.reshape(-1, 1))

    # Calculate metrics
    mae = mean_absolute_error(y_test_true_inv, y_test_pred)
    mse = mean_squared_error(y_test_true_inv, y_test_pred)
    mape = mean_absolute_percentage_error(y_test_true_inv, y_test_pred)

    # Plot the results


    # Return results
    return {
        'model': model,
        'predictions': y_test_pred,
        'actual': y_test_true_inv,
        'metrics': {
            'MAE': mae,
            'MSE': mse,
            'MAPE': mape
        }
    }
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential

def predict_future(model, series, scaler, seq_length, n_future=20):
    """
    Make future predictions using the trained LSTM model

    Parameters:
    model (Sequential): Trained Keras LSTM model
    series (pd.Series): Original time series data
    scaler (MinMaxScaler): Fitted scaler used for the original data
    seq_length (int): Sequence length used for training
    n_future (int): Number of future time steps to predict

    Returns:
    np.array: Array of future predictions
    """
    # Get the most recent data points as the initial input
    data = series.values.reshape(-1, 1)
    last_sequence = data[-seq_length:].reshape(1, seq_length, 1)

    # Scale the sequence
    last_sequence_scaled = scaler.transform(last_sequence.reshape(seq_length, 1)).reshape(1, seq_length, 1)

    # Initialize array to store predictions
    future_predictions = []

    # Make iterative predictions
    curr_sequence = last_sequence_scaled.copy()

    for _ in range(n_future):
        # Get prediction (scaled)
        pred_scaled = model.predict(curr_sequence, verbose=0)[0]

        # Store the prediction
        future_predictions.append(scaler.inverse_transform(pred_scaled.reshape(1, -1))[0, 0])

        # Update the sequence by removing the first element and adding the prediction
        curr_sequence = np.append(curr_sequence[:, 1:, :],
                                 [[pred_scaled]],
                                 axis=1)

    return np.array(future_predictions)

def plot_with_future_predictions(series, future_predictions):
    """
    Plot the original series with future predictions

    Parameters:
    series (pd.Series): Original time series data
    future_predictions (np.array): Array of future predictions
    """
    plt.figure(figsize=(14, 7))

    # Plot original data
    plt.plot(range(len(series)), series.values, 'b-', label='Historical Data')

    # Plot future predictions
    plt.plot(range(len(series)-1, len(series)+len(future_predictions)-1),
             future_predictions, 'r--', label='Future Predictions')

    # Add a vertical line to separate historical and future data
    plt.axvline(x=len(series)-1, color='k', linestyle='-', alpha=0.2)
    plt.text(len(series), min(series), 'Forecast Start', fontsize=12)

    # Add labels and legend
    plt.title('Time Series Forecast - 20 Future Steps', fontsize=16)
    plt.xlabel('Time Steps', fontsize=12)
    plt.ylabel('Value', fontsize=12)
    plt.legend(loc='best')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()



# Example usage with the previously created model


In [ ]:
mae_list = []
mape_list = []
mse_list = []
df = pd.read_excel('/content/all_meandata-50.xlsx').iloc[:,1:]
for columnn in df.columns[1400:1531]:
  zone = df[columnn]
  if __name__ == "__main__":
      # Import the functions from the previous code


      # Create a sample time series
      np.random.seed(42)
      dates = pd.date_range(start='2023-01-01', periods=30, freq='D')
      values = np.sin(np.linspace(0, 4*np.pi, 30)) + np.random.normal(0, 0.1, 30)
      sample_series = pd.Series(zone)

      # Define sequence length used in model training
      seq_length = 3

      # Prepare scaler and scale data
      scaler = MinMaxScaler(feature_range=(0, 1))
      scaled_data = scaler.fit_transform(zone.values.reshape(-1, 1))

      # Build LSTM model with our helper function (or load your existing model)
      #results = build_lstm_model(zone)
      #trained_model = results['model']

      # Make future predictions
      #future_values = predict_future(trained_model, zone, scaler, seq_length, n_future=20)

      # Plot original data with future predictions
      #plot_with_future_predictions(zone, future_values)
      #plt.savefig('/content/'+columnn+'_fig_10_years.png')
      #plt.close('all')
      # Print predictions




      sample_series = pd.Series(zone)

      # Build and evaluate the LSTM model
      results = build_lstm_model(sample_series)

      # Print the metrics

      mae_list.append(results['metrics']['MAE'])
      mse_list.append(results['metrics']['MSE'])
      mape_list.append(results['metrics']['MAPE'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a l

In [ ]:
pd.DataFrame({'zone':df.columns[1400:1531],'mae_test':mae_list,'mape_test':mape_list,'mse_test':mse_list}).to_excel('/content/all_lstm_errors_data_1400-1531.xlsx')

In [ ]:
import os
path = '/content/all_lstm'
listed = os.listdir(path)
df = pd.DataFrame()
for e in listed:
  df = pd.concat([df , pd.read_excel(path + '/' + e)] , axis = 0 , ignore_index=True)

In [ ]:
df.iloc[:,1:].to_excel('/content/lstm_errors.xlsx')